In [13]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [14]:
jobs = pd.read_csv('saco2.csv')
jobs.set_index('clock', inplace=True)
jobs.drop(['Unnamed: 0'], axis=1, inplace=True)

In [15]:
jobs

,job_id,tipo,instante
clock,,,
20,1,Chegada,20
20,2,Chegada,20
20,1,Ingresso,0
20,2,Ingresso,0
20,1,ReqDisp,0
...,...,...,...
385,4,SaidaSis,0
370,3,FimProcess,0
370,3,LiberaProcess,0


In [16]:
def plot_states_by_job_id(jobs, job_id):

    job_id_data = jobs[jobs.job_id == job_id]

    trace = go.Scatter(x=job_id_data.index,
                       y=job_id_data.tipo,
                       mode='lines+markers',
                       name='job='+str(job_id))

    return trace


fig = make_subplots(specs=[[{"secondary_y": True}]])

for job_id in jobs.job_id.unique():

    fig_trace = plot_states_by_job_id(jobs, job_id)
    fig.add_trace(fig_trace, secondary_y=False)


order = ['Chegada', 'Ingresso', 'ReqDisp', 'ReqMem', 'ReqProcess', 'Executando', 'FimProcess', 'LiberaProcess',
         'LiberaMem', 'SaidaSis', 'Fim']
fig.update_layout(title="Estados dos jobs ao longo do tempo",
                  plot_bgcolor='white', yaxis={'categoryarray': order, 'categoryorder': "array"})

multiprog_on = []
for idx, clock in zip(range(len(jobs.index.unique())), jobs.index.unique()):
    jobs_exec = jobs[(jobs.tipo == "Executando") & (jobs.index == clock)]

    if len(jobs.index.unique()) != idx + 1:
        next_clock = jobs.index.unique()[idx + 1]
    else:
        continue

    if jobs_exec.shape[0] <= 1:
        multiprog_on.append('Execução única')

        if next_clock - clock  > 1:
            for clk in range(clock, next_clock):
                multiprog_on.append('Execução única')

    else:
        multiprog_on.append('Execução multipla')
        last_state = 'Execução multipla'

        if next_clock - clock  > 1:
            for clk in range(clock, next_clock):
                multiprog_on.append('Execução única')

In [17]:
is_multiprog = pd.DataFrame(
    multiprog_on, columns=['is_multiprog'])

# # fig = go.Figure()

trace = go.Scatter(x=is_multiprog.index+20,
                   y=is_multiprog.is_multiprog,
                   fill='tozeroy',
                   line_color='lightgray',
                   name='Execução Multipla')
fig.update_layout(yaxis2={'categoryarray': [
                  'Execução única', 'Execução multipla'], 'categoryorder': "array"})

fig.add_trace(trace, secondary_y=True)

fig.update_xaxes(title="Clock", dtick=50)
fig.update_yaxes(title="Estado Atual")

In [19]:
simulation = pd.read_json('data/multi/multiprog_macro_data.json')
simulation.set_index('clock', inplace=True)

fig = go.Figure()

trace = go.Scatter(x=simulation.index,
                   y=simulation.memoria * 1000)
fig.add_trace(trace)
fig.update_xaxes(range=[0, simulation.index[-1] + 2])

fig.update_xaxes(title="Clock")
fig.update_yaxes(title="Memória", dtick=10000)

fig.update_layout(title="Utilização da memória ao longo do tempo")
fig.write_image('data/multi/3memoria_multiprog.png')

fig = go.Figure()

trace = go.Scatter(x=simulation.index,
                   y=simulation.disco+30)
fig.add_trace(trace)
fig.update_xaxes(range=[0, simulation.index[-1] + 2])

fig.update_xaxes(title="Clock")
fig.update_yaxes(title="Disco")

fig.update_layout(title="Armazenamento dos Jobs e Arquivos no disco ao longo do tempo")